In [1]:
import numpy as np

from angle_emb import AnglE, Prompts
angle = AnglE.from_pretrained('BAAI/bge-large-en-v1.5').cuda()
angle.set_prompt(prompt=Prompts.C)

def getEmbeddingsFunction(txt):
    return angle.encode({'text': txt}, to_numpy=True)[0].tolist()
#     return  angle.encode(txt)[0].tolist()

def get_embeddings(texts):
    embeddings = []
    for text in texts:
        # Replace this with your actual getEmbeddings function
        embedding = getEmbeddingsFunction(text)
        embeddings.append(embedding)
    return list(embeddings)


INFO:AnglE:Prompt is set, the prompt will be automatically applied during the encoding phase. To disable prompt setting, please configure set_prompt(prompt=None)


In [ ]:
from transformers import GPT2Tokenizer

def count_tokens(text):
    # Load the tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

    # Tokenize the text
    tokens = tokenizer.encode(text)

    # Return the number of tokens
    return len(tokens)

# Example text
text = "Hello, world! This is an example to count tokens."
num_tokens = count_tokens(text)
print(f"Number of tokens: {num_tokens}")


In [2]:
import chromadb
from chromadb.config import Settings

chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
chroma_client = chromadb.PersistentClient(path="./chromaDB")



INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [3]:
collection = chroma_client.get_collection(name="ee_help")
collection.count()

1188

In [123]:
queryText = "is any country affected"
query = getEmbeddingsFunction(queryText)

In [124]:
resultsRules = collection.query(
    query_embeddings=[query],
    n_results=20
)

In [125]:
resultsRules['documents']

[['URL: ee.co.uk/help/mobile/roaming/eu-roaming-charges.html, Heading: Charges for roaming in the EU, Question: Which countries will be affected?, Answer: \nThe full list of countries affected by the EU roaming charges: Austria, Azores, Belgium, Bulgaria, Croatia, Cyprus (excludes northern Cyprus), Czech Republic, Canary Islands, Denmark, Estonia, Finland, France, French Guiana, Germany, Gibraltar, Greece, Guadeloupe, Guernsey, Hungary, Iceland, Isle of Man, Italy, Jersey, Latvia, Lichtenstein, Lithuania, Luxembourg, Madeira, Malta, Martinique, Mayotte, Monaco, Netherlands, Norway, Poland, Portugal, Reunion Islands, Romania, San Marino, Saint Martin (French), Saint Barthelemy, Slovakia, Slovenia, Spain, Sweden, Switzerland and Vatican City (Italy).\nCalls, texts and data within the Republic of Ireland are included within your allowance. If you use up your allowance, the\xa0£2.29 daily charge will apply.\n',
  'URL: ee.co.uk/help/mobile/fix-problem/emergency-alert-explained.html, Headin

In [110]:
queryText = "Can i use my plan in Sapin"
query = getEmbeddingsFunction(queryText)

In [111]:
results = collection.query(
    query_embeddings=[query],
    n_results=20
)

In [112]:
print('''You are an Help Desk Agent for a company called EE who answers customers queries regarding EE's services
Answer briefly but fully and to the point the following question with information provided in Sources section, and rules section
Use only the sources to form your answer
The rules section should be used to mention any changes / aditional charges that would apply to the customer
Include any charges,plans etc or extra information the customer should be made aware of
If the customer needs to send any sms or visit any pages, include it in the answer
List the urls of sources used for generating the answer (leave 2 lines before this section)
''')
print('\n')

print('Question: ', queryText)
print('\n')

print('Rules:')
print(resultsRules['documents'][0])
print('\n')

print('Sources:')
print(results['documents'][0])

You are an Help Desk Agent for a company called EE who answers customers queries regarding EE's services
Answer briefly but fully and to the point the following question with information provided in Sources section, and rules section
Use only the sources to form your answer
The rules section should be used to mention any changes / aditional charges that would apply to the customer
Include any charges,plans etc or extra information the customer should be made aware of
If the customer needs to send any sms or visit any pages, include it in the answer
List the urls of sources used for generating the answer (leave 2 lines before this section)



Question:  Can i use my plan in Sapin


Rules:
['URL: ee.co.uk/help/billing-payments/payment-methods/flex-plans-explained.html, Heading: Flex plans explained, Question: What if my allowances run out before my plan renews?, Answer: \nIf you’ve used some of your allowances before your plan renews:\nYou can buy an add-on by logging in to EE\nIf you’ve